In [1]:
import cv2
import numpy as np
import pyautogui
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import Flatten,Dense
from keras.preprocessing.image import ImageDataGenerator
import subprocess
import os
import webbrowser
import ctypes
import tensorflow as tf
pyautogui.FAILSAFE=False

Using TensorFlow backend.


In [4]:
hand=os.listdir("hand/train")
hand

['fist', 'five', 'four', 'one', 'three', 'two']

In [3]:
classifier=Sequential()

In [157]:
classifier.add(Conv2D(32,(5,5),input_shape=(64,64,3),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Conv2D(64,(5,5),activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(units=1042,activation='relu'))
classifier.add(Dense(units=216,activation='relu'))
classifier.add(Dense(units=36,activation='relu'))
classifier.add(Dense(units=6,activation='softmax'))
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [159]:
train_dataset=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [160]:
train_set=train_dataset.flow_from_directory(
    'hand/train',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

Found 1042 images belonging to 6 classes.


In [161]:
test_set=train_dataset.flow_from_directory(
    'hand/test',
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

Found 174 images belonging to 6 classes.


In [162]:
classifier.fit_generator(
        train_set,
        steps_per_epoch=1042,
        epochs=5,
        validation_data=test_set,
        validation_steps=174)

Epoch 1/5
1042/1042 [==============================] - 649s 623ms/step - loss: 0.0582 - acc: 0.9807 - val_loss: 0.9974 - val_acc: 0.8805
Epoch 2/5
1042/1042 [==============================] - 643s 617ms/step - loss: 0.0048 - acc: 0.9986 - val_loss: 0.9338 - val_acc: 0.8900
Epoch 3/5
1042/1042 [==============================] - 638s 612ms/step - loss: 0.0081 - acc: 0.9985 - val_loss: 1.5733 - val_acc: 0.8064
Epoch 4/5
1042/1042 [==============================] - 644s 618ms/step - loss: 0.0300 - acc: 0.9954 - val_loss: 1.1162 - val_acc: 0.8864
Epoch 5/5
1042/1042 [==============================] - 641s 615ms/step - loss: 0.0019 - acc: 0.9998 - val_loss: 1.2609 - val_acc: 0.8785


In [163]:
classifier.save('hand/hand.h5')

In [2]:
classifier=tf.keras.models.load_model('hand/hand.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
hand

['fist', 'five', 'four', 'one', 'three', 'two']

In [6]:
def pred():
    import numpy as np
    from keras.preprocessing import image

    import numpy as np
    from keras.preprocessing import image
    test_image = image.load_img('hand/a.jpg', target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    guess=result.argmax()
    return (hand[guess])


In [7]:
def inc(value):
    for i in g:
        if value==i:
            g[i]+=1
            if g[i]>=5:
                print(g)
                g[i]=0
                gesture(i)
                
        else:
            g[i]=0


In [8]:
def gesture(value):
    if value=='one':
        subprocess.Popen([("C:/Program Files (x86)/VideoLAN/VLC/vlc.exe")])
    elif value=='two':
        pyautogui.moveTo(c[0][0][0]*3,c[0][0][1]*3, duration = 0.1) 
    elif value=='three':
        webbrowser.get("C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s").open("http://google.com")
    elif value=='four':
        webbrowser.get("C:/Program Files (x86)/Google/Chrome/Application/chrome.exe %s").open("http://youtube.com")
    elif value=='five':
        ctypes.windll.user32.LockWorkStation()
    elif value=='fist':
        pyautogui.click(x=1920,y=0,button='right')

In [9]:
def preprocess(action_frame):
    
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
  
    blur = cv2.GaussianBlur(action_frame, (5,5), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_RGB2HSV)

    lower_color = np.array([108, 23, 82])
    upper_color = np.array([179, 255, 255])
    for (x,y,w,h) in faces:
        hsv[y+2:y+h-2,x+2:x+w-2]=img[y+2:y+h-2,x+2:x+w-2]
    mask = cv2.inRange(hsv, lower_color, upper_color)
    blur = cv2.medianBlur(mask, 5)

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8, 8))
    hsv_d = cv2.dilate(blur, kernel)
    return hsv_d

In [11]:
g={'fist':0, 'five':0, 'four':0, 'one':0, 'three':0, 'two':0}
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
img = np.zeros([1920,1080,1],dtype=np.uint8)
cap=cv2.VideoCapture(0)
cv2.namedWindow("output", cv2.WINDOW_NORMAL) 
cv2.resizeWindow("output", 1920, 1080)
run=0
while True:
    _,frame=cap.read()
    run=run+1
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    
    hsv=preprocess(frame)
    contours,_=cv2.findContours(hsv,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    
    n=len(contours)
    contours=sorted(contours,key=cv2.contourArea,reverse=False)[:n]
    for c in contours:
        hull=cv2.convexHull(c)
        area=cv2.contourArea(c)
        if area >5000:
            cv2.drawContours(frame,[hull],0,(0,255,0),2)
            cv2.imwrite("hand/a.jpg",hsv)
            name=pred()
            inc(name)
            cv2.putText(frame, name, (400,100), cv2.FONT_HERSHEY_SIMPLEX,3, (0,0,255), 2, cv2.LINE_AA)
            pyautogui.moveTo(c[0][0][0]*3,c[0][0][1]*3, duration = 0.1) 
    
    cv2.imshow("output",frame)
    
    cv2.imshow("mask",hsv)
    if cv2.waitKey(100) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

{'fist': 5, 'five': 0, 'four': 0, 'one': 0, 'three': 0, 'two': 0}
{'fist': 0, 'five': 0, 'four': 5, 'one': 0, 'three': 0, 'two': 0}
